#### Part 15: SAS, SPSS Formats and Random Sampling in Pandas

In this notebook, we'll explore:
- Working with SAS and SPSS file formats
- Performance considerations in pandas
- Random sampling from Series and DataFrames

##### Setup
First, let's import the necessary libraries:

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import os

##### 1. SAS Formats

The top-level function `read_sas()` can read (but not write) SAS xport (.XPT) and SAS7BDAT (.sas7bdat) format files.

In [2]:
# Read a SAS7BDAT file (example - you would need an actual SAS file)
'''
df = pd.read_sas('sas_data.sas7bdat')
'''

"\ndf = pd.read_sas('sas_data.sas7bdat')\n"

In [3]:
# Obtain an iterator and read an XPORT file 100,000 lines at a time
'''
def do_something(chunk):
    pass

rdr = pd.read_sas('sas_xport.xpt', chunk=100000)
for chunk in rdr:
    do_something(chunk)
'''

"\ndef do_something(chunk):\n    pass\n\nrdr = pd.read_sas('sas_xport.xpt', chunk=100000)\nfor chunk in rdr:\n    do_something(chunk)\n"

##### 2. SPSS Formats

The top-level function `read_spss()` can read (but not write) SPSS sav (.sav) and zsav (.zsav) format files.

In [4]:
# Read an SPSS file (example - you would need an actual SPSS file)
'''
df = pd.read_spss('spss_data.sav')
'''

"\ndf = pd.read_spss('spss_data.sav')\n"

In [5]:
# Extract a subset of columns and avoid converting categorical columns
'''
df = pd.read_spss('spss_data.sav', usecols=['foo', 'bar'],
                  convert_categoricals=False)
'''

"\ndf = pd.read_spss('spss_data.sav', usecols=['foo', 'bar'],\n                  convert_categoricals=False)\n"

##### 3. Performance Considerations

Let's create a sample DataFrame for performance testing:

In [6]:
sz = 1000000
df = pd.DataFrame({'A': np.random.randn(sz), 'B': [1] * sz})

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   A       1000000 non-null  float64
 1   B       1000000 non-null  int64  
dtypes: float64(1), int64(1)
memory usage: 15.3 MB


In [7]:
# Example functions for testing IO performance
def test_sql_write(df):
    if os.path.exists('test.sql'):
        os.remove('test.sql')
    sql_db = sqlite3.connect('test.sql')
    df.to_sql(name='test_table', con=sql_db)
    sql_db.close()

def test_sql_read():
    sql_db = sqlite3.connect('test.sql')
    pd.read_sql_query("select * from test_table", sql_db)
    sql_db.close()

def test_hdf_fixed_write(df):
    df.to_hdf('test_fixed.hdf', 'test', mode='w')

##### 4. Selecting Random Samples

A random selection of rows or columns from a Series or DataFrame can be obtained with the `sample()` method. The method will sample rows by default, and accepts a specific number of rows/columns to return, or a fraction of rows.

In [8]:
s = pd.Series([0, 1, 2, 3, 4, 5])

# When no arguments are passed, returns 1 row
s.sample()

4    4
dtype: int64

In [9]:
# Specify a number of rows
s.sample(n=3)

2    2
1    1
0    0
dtype: int64

In [10]:
# Or a fraction of the rows
s.sample(frac=0.5)

4    4
5    5
2    2
dtype: int64

### 4.1 Sampling with Replacement

By default, `sample` will return each row at most once, but one can also sample with replacement using the `replace` option:

In [11]:
s = pd.Series([0, 1, 2, 3, 4, 5])

# Without replacement (default)
s.sample(n=6, replace=False)

5    5
3    3
0    0
4    4
1    1
2    2
dtype: int64

In [12]:
# With replacement
s.sample(n=6, replace=True)

3    3
1    1
0    0
3    3
1    1
3    3
dtype: int64

### 4.2 Sampling with Weights

By default, each row has an equal probability of being selected, but if you want rows to have different probabilities, you can pass the sample function sampling weights as `weights`.

In [13]:
s = pd.Series([0, 1, 2, 3, 4, 5])

example_weights = [0, 0, 0.2, 0.2, 0.2, 0.4]

s.sample(n=3, weights=example_weights)

2    2
4    4
3    3
dtype: int64

In [14]:
# Weights will be re-normalized automatically
example_weights2 = [0.5, 0, 0, 0, 0, 0]

s.sample(n=1, weights=example_weights2)

0    0
dtype: int64

### 4.3 Using DataFrame Column as Weights

When applied to a DataFrame, you can use a column of the DataFrame as sampling weights (provided you are sampling rows and not columns) by simply passing the name of the column as a string.

In [15]:
df2 = pd.DataFrame({'col1': [9, 8, 7, 6],
                   'weight_column': [0.5, 0.4, 0.1, 0]})

df2.sample(n=3, weights='weight_column')

,col1,weight_column
0,9,0.5
1,8,0.4
2,7,0.1


### 4.4 Sampling Columns

The `sample` method also allows users to sample columns instead of rows using the `axis` argument.

In [16]:
df3 = pd.DataFrame({'col1': [1, 2, 3], 'col2': [2, 3, 4]})

# Sample columns instead of rows
df3.sample(n=1, axis=1)

,col2
0,2
1,3
2,4
